In [ ]:
# Quick exploration of the files in the provided folder

# Import the Google Cloud Storage client library
from google.cloud import storage
import os # Often useful, though not strictly needed for this specific script

# --- Configuration ---
# Extract the bucket name and the prefix from the gs:// URI
# The bucket is everything between gs:// and the first slash
# The prefix is everything after the first slash (including it, to denote the path)
bucket_name = 'labs.roitraining.com'
# The prefix is like a directory path within the bucket.
# Adding the trailing slash '/' typically tells GCS to list objects *within* this path structure.
prefix = 'alaska-dept-of-snow/'

# --- Script ---
print(f"Attempting to list contents of gs://{bucket_name}/{prefix}...")

try:
    # Instantiate a storage client.
    # This automatically uses Application Default Credentials (ADC)
    # when running in a GCP environment with appropriate permissions.
    storage_client = storage.Client()

    # List blobs (objects) in the specified bucket with the given prefix.
    # The list_blobs method is efficient for iterating through objects matching a prefix.
    # It handles pagination automatically for large numbers of files.
    blobs_iterator = storage_client.list_blobs(bucket_name, prefix=prefix)

    print("\nContents found:")
    found_content = False
    # Iterate through the iterator and print each blob's name
    for blob in blobs_iterator:
        # The blob.name is the full path of the object within the bucket
        print(f"- {blob.name}")
        found_content = True

    if not found_content:
        print("No objects found at this location, or the location does not exist within the bucket.")
        print("(Note: GCS folders are virtual; this lists objects whose names start with the prefix.)")


except Exception as e:
    print(f"\nAn error occurred: {e}")
    print("\nPlease check:")
    print("1. That your GCP environment (e.g., the service account for your notebook) has the 'storage.objects.list' permission on the bucket.")
    print("2. That the bucket name ('labs.roitraining.com') and prefix ('alaska-dept-of-snow/') are correct.")
    print("3. Your internet connection.")


print("\nListing attempt complete.")

Attempting to list contents of gs://labs.roitraining.com/alaska-dept-of-snow/...

Contents found:
- alaska-dept-of-snow/.DS_Store
- alaska-dept-of-snow/alaska-dept-of-snow-faqs.csv
- alaska-dept-of-snow/faq-01.txt
- alaska-dept-of-snow/faq-02.txt
- alaska-dept-of-snow/faq-03.txt
- alaska-dept-of-snow/faq-04.txt
- alaska-dept-of-snow/faq-05.txt
- alaska-dept-of-snow/faq-06.txt
- alaska-dept-of-snow/faq-07.txt
- alaska-dept-of-snow/faq-08.txt
- alaska-dept-of-snow/faq-09.txt
- alaska-dept-of-snow/faq-10.txt
- alaska-dept-of-snow/faq-11.txt
- alaska-dept-of-snow/faq-12.txt
- alaska-dept-of-snow/faq-13.txt
- alaska-dept-of-snow/faq-14.txt
- alaska-dept-of-snow/faq-15.txt
- alaska-dept-of-snow/faq-16.txt
- alaska-dept-of-snow/faq-17.txt
- alaska-dept-of-snow/faq-18.txt
- alaska-dept-of-snow/faq-19.txt
- alaska-dept-of-snow/faq-20.txt
- alaska-dept-of-snow/faq-21.txt
- alaska-dept-of-snow/faq-22.txt
- alaska-dept-of-snow/faq-23.txt
- alaska-dept-of-snow/faq-24.txt
- alaska-dept-of-snow/faq-2

In [ ]:
# Quick exploration of the content in the first several files

# Import the Google Cloud Storage client library
from google.cloud import storage
from google.api_core import exceptions # Import for specific error types

# --- Configuration ---
bucket_name = 'labs.roitraining.com'

# List of the specific object names you want to preview
# Taken directly from your output list (the first 5)
object_names_to_preview = [
    'alaska-dept-of-snow/.DS_Store',
    'alaska-dept-of-snow/alaska-dept-of-snow-faqs.csv',
    'alaska-dept-of-snow/faq-01.txt',
    'alaska-dept-of-snow/faq-02.txt',
    'alaska-dept-of-snow/faq-03.txt',
]

preview_limit = 1000 # Limit the number of characters to print for each preview

# --- Script ---
print(f"Attempting to preview the content of the first {len(object_names_to_preview)} files...")

try:
    # Instantiate a storage client (uses ADC)
    storage_client = storage.Client()

    # Loop through the specified object names and print content
    for object_name in object_names_to_preview:
        print("\n" + "=" * 60) # Separator for clarity
        print(f"Previewing: gs://{bucket_name}/{object_name}")
        print("=" * 60)

        # Get a reference to the blob (object) directly without calling get_bucket() first
        # This approach doesn't require storage.buckets.get permission
        blob = storage_client.bucket(bucket_name).blob(object_name)

        try:
            # Check if the blob exists before trying to download
            # This check *also* requires permissions, often storage.objects.get or storage.objects.list
            # If this fails, the download will likely also fail, but this check is cleaner.
            # In some strict permission scenarios, checking existence might require a different permission than reading.
            # Let's proceed directly to download and handle potential errors there as well.
            # if not blob.exists():
            #     print("  [Info] File not found or you lack permission to check its existence.")
            #     continue

            # Download the blob's content as a string.
            # This explicitly requires storage.objects.get permission on *this specific object*.
            content = blob.download_as_string().decode('utf-8')

            # Print limited preview
            if len(content) > preview_limit:
                print("--- Content (first {} characters) ---".format(preview_limit))
                print(content[:preview_limit] + "\n...") # Add ellipsis to show it's truncated
                print("------------------------------------")
            else:
                print("--- Content ---")
                print(content)
                print("---------------")

        except exceptions.NotFound:
             print("  [Error] File not found or you lack permission to read it.")
        except UnicodeDecodeError:
            print("  [Error] Could not decode file content as UTF-8. It might be a binary file (.DS_Store is likely binary).")
        except exceptions.PermissionDenied:
             print(f"  [Error] Permission denied to read object: {object_name}. You need 'storage.objects.get'.")
        except Exception as e:
            print(f"  [Error] An unexpected error occurred while reading {object_name}: {e}")

except Exception as e:
    # This outer catch block would only trigger if storage.Client() itself fails
    # or if something goes wrong *before* the loop starts, which is less likely now.
    print(f"\nAn error occurred during client setup: {e}")


print("\nPreview attempt complete.")

Attempting to preview the content of the first 5 files...

Previewing: gs://labs.roitraining.com/alaska-dept-of-snow/.DS_Store
  [Error] Could not decode file content as UTF-8. It might be a binary file (.DS_Store is likely binary).

Previewing: gs://labs.roitraining.com/alaska-dept-of-snow/alaska-dept-of-snow-faqs.csv
--- Content (first 1000 characters) ---
"question","answer"
"When was the Alaska Department of Snow established?","The Alaska Department of Snow (ADS) was established in 1959, coinciding with Alaska’s admission as a U.S. state."
"What is the mission of the Alaska Department of Snow?","Our mission is to ensure safe, efficient travel and infrastructure continuity by coordinating snow removal services across the state’s 650,000 square miles."
"How does ADS coordinate plowing across different regions?","ADS works with local municipalities and regional offices to schedule and prioritize plowing routes, focusing first on high-traffic roads, emergency routes, and schools."
"Who

In [ ]:
# Loading the data to BQ from the CSV, after validating the CSV contains the same info as the txt files

# Script to load FAQ CSV data to BigQuery for use in RAG runtime (using genai_app dataset)

from google.cloud import bigquery
import os
import re

# --- Configuration ---
# GCS URI for the source CSV file
gcs_uri = "gs://labs.roitraining.com/alaska-dept-of-snow/alaska-dept-of-snow-faqs.csv"

# *** Updated Dataset ID ***
dataset_id = "genai_app"

# Generate table ID from the filename part of the GCS URI
# Extract the filename (e.g., 'alaska-dept-of-snow-faqs.csv')
filename = gcs_uri.split('/')[-1]

# Remove the .csv extension (e.g., 'alaska-dept-of-snow-faqs')
table_name_base = filename.rsplit('.', 1)[0]

# Replace hyphens with underscores and convert to lowercase for BQ table ID
table_id = table_name_base.replace('-', '_').lower() # -> 'alaska_dept_of_snow_faqs'

# Use your Qwiklabs project ID
# Ensure this is the correct project where you want the dataset and table to exist
project_id = "qwiklabs-gcp-03-28c3125acb2b" # <<< VERIFY THIS IS YOUR CURRENT PROJECT ID

# --- Initialize the BigQuery client ---
# Initialize the client with your project ID.
# This uses Application Default Credentials (ADC) from your environment.
try:
    client = bigquery.Client(project=project_id)
    print(f"BigQuery client initialized for project: {client.project}")
except Exception as e:
    print(f"Error initializing BigQuery client: {e}")
    print("Please ensure you are running in a GCP environment with appropriate credentials.")
    exit() # Exit if client cannot be initialized

# Construct the full table ID string (format: project.dataset.table)
table_id_full = f"{client.project}.{dataset_id}.{table_id}"

# --- Load Job Configuration ---
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Assuming your CSV has a header row to skip
    # *** Explicitly define the schema ***
    # This overrides autodetect and ensures columns are 'question' and 'answer'
    schema=[
        bigquery.SchemaField("question", "STRING"),
        bigquery.SchemaField("answer", "STRING"),
    ],
    # *** How to write data ***
    # WRITE_TRUNCATE: If the table exists, overwrite the data. If it doesn't, create it.
    # WRITE_APPEND: If the table exists, append the data. If it doesn't, create it.
    # WRITE_EMPTY: If the table exists and contains data, the job fails. If it's empty or doesn't exist, create it.
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

# --- Start the Load Job ---
print(f"\nStarting load job for '{gcs_uri}' into '{table_id_full}'...")

try:
    load_job = client.load_table_from_uri(
        gcs_uri,
        table_id_full,
        job_config=job_config,
    )  # API request

    print(f"Load job {load_job.job_id} initiated. Waiting for job to complete...")

    # Wait for the job to complete. result() blocks until the job is done.
    load_job.result()

    print("Load job finished successfully.")

    # Verify the load by getting the destination table and checking row count
    try:
        destination_table = client.get_table(table_id_full)
        print(f"Successfully loaded {destination_table.num_rows} rows into {table_id_full}.")
    except Exception as e:
        print(f"Warning: Load job finished, but could not get table details after load: {e}")

except Exception as e:
    print(f"\nAn error occurred during the load job: {e}")
    print("\nPlease check:")
    print(f"1. That the GCS URI '{gcs_uri}' is correct and accessible.")
    print(f"2. That the BigQuery dataset '{dataset_id}' exists in project '{client.project}'.")
    print("3. That the service account running this notebook has the necessary permissions:")
    print("   - storage.objects.get on the GCS file.")
    print(f"   - bigquery.tables.create and bigquery.tables.updateData on the '{dataset_id}' dataset in your project.")
    print("4. The format of the CSV file matches the expected two columns ('question', 'answer') and is valid CSV.")


print("\nBigQuery load script complete.")

BigQuery client initialized for project: qwiklabs-gcp-03-28c3125acb2b

Starting load job for 'gs://labs.roitraining.com/alaska-dept-of-snow/alaska-dept-of-snow-faqs.csv' into 'qwiklabs-gcp-03-28c3125acb2b.genai_app.alaska_dept_of_snow_faqs'...
Load job 1f893e8e-2485-416b-8db5-e37bdb6deb6e initiated. Waiting for job to complete...
Load job finished successfully.
Successfully loaded 50 rows into qwiklabs-gcp-03-28c3125acb2b.genai_app.alaska_dept_of_snow_faqs.

BigQuery load script complete.


In [ ]:
# Script to create BigQuery ML model and embeddings table using Python (Update from previous method of running directly in BQ interface)

from google.cloud import bigquery
from google.api_core import exceptions

# --- Configuration ---
# Your Qwiklabs project ID
project_id = "qwiklabs-gcp-03-28c3125acb2b"

# Your target dataset
dataset_id = "genai_app"

# Source table where your FAQ data is
source_table_id = "alaska_dept_of_snow_faqs"

# BigQuery ML model name
model_id = "embeddings_model"

# Name for the new table that will contain the embeddings
# Convention: original_table_name_with_embeddings
embeddings_table_id = f"{source_table_id}_with_embeddings" # -> 'alaska_dept_of_snow_faqs_with_embeddings'

# Vertex AI connection name and location
connection_id = "us.vertex-rag-connection"

# Vertex AI text embedding model endpoint
vertex_ai_endpoint = "text-embedding-005"

# --- Initialize the BigQuery client ---
try:
    client = bigquery.Client(project=project_id)
    print(f"BigQuery client initialized for project: {client.project}")
except Exception as e:
    print(f"Error initializing BigQuery client: {e}")
    print("Please ensure you are running in a GCP environment with appropriate credentials.")
    exit()

# Step 1: Create/Replace BigQuery ML Remote Model
# Construct the full model ID string (format: project.dataset.model)
model_id_full = f"{client.project}.{dataset_id}.{model_id}"

# SQL query to create the remote model
create_model_sql = f"""
CREATE OR REPLACE MODEL `{model_id_full}`
REMOTE WITH CONNECTION `{connection_id}`
OPTIONS (ENDPOINT = '{vertex_ai_endpoint}');
"""

print(f"\nAttempting to create/replace BigQuery ML model: {model_id_full}")
print(f"Using connection: {connection_id}")
print(f"Pointing to Vertex AI endpoint: {vertex_ai_endpoint}")

try:
    # Execute the SQL query as a BigQuery job
    query_job_model = client.query(create_model_sql)

    print(f"Model creation job {query_job_model.job_id} initiated. Waiting for job to complete...")

    # Wait for the job to complete. result() blocks until the job is done.
    query_job_model.result()

    print(f"BigQuery ML model '{model_id}' created/replaced successfully in dataset '{dataset_id}'.")

except exceptions.NotFound:
    print(f"\nError: Connection '{connection_id}' not found.")
    print("Please ensure the connection exists and is in the correct location.")
except exceptions.PermissionDenied:
    print(f"\nError: Permission denied to create model or use connection.")
    print("Ensure the service account has 'bigquery.models.create' and 'bigquery.connections.use' permissions.")
except Exception as e:
    print(f"\nAn error occurred during model creation: {e}")
    print("Please check the connection name, location, endpoint, and permissions.")


# --- Step 2: Create/Replace Embeddings Table ---
# Construct the full source and destination table IDs
source_table_id_full = f"{client.project}.{dataset_id}.{source_table_id}"
embeddings_table_id_full = f"{client.project}.{dataset_id}.{embeddings_table_id}"

# SQL query to create the embeddings table
# It uses ML.GENERATE_EMBEDDING with the model created above
# The inner query selects the data and concatenates question and answer into a 'content' column for embedding
# SELECT * EXCEPT(content) selects all columns from the ML.GENERATE_EMBEDDING output *except* the intermediate 'content' column
create_embeddings_table_sql = f"""
CREATE OR REPLACE TABLE `{embeddings_table_id_full}` AS
SELECT * EXCEPT(content)
FROM ML.GENERATE_EMBEDDING(
    MODEL `{model_id_full}`,
    (SELECT question, answer, CONCAT(question, answer) AS content FROM `{source_table_id_full}`)
);
"""

print(f"\nAttempting to create/replace embeddings table: {embeddings_table_id}")
print(f"Using source data from: {source_table_id_full}")
print(f"Using model: {model_id_full}")

try:
    # Execute the SQL query as a BigQuery job
    query_job_embedding = client.query(create_embeddings_table_sql)

    print(f"Embeddings table creation job {query_job_embedding.job_id} initiated. Waiting for job to complete...")

    # Wait for the job to complete.
    # This job interacts with Vertex AI, so it might take longer than the model creation.
    query_job_embedding.result()

    print(f"Embeddings table '{embeddings_table_id}' created/replaced successfully in dataset '{dataset_id}'.")

    # Optional: Verify row count in the new table
    try:
        destination_table = client.get_table(embeddings_table_id_full)
        print(f"Successfully created embeddings table with {destination_table.num_rows} rows.")
        # print("Schema of the new table:")
        # for field in destination_table.schema:
        #     print(f"- {field.name}: {field.field_type}")
    except Exception as e:
         print(f"Warning: Embeddings job finished, but could not get destination table details: {e}")


except exceptions.NotFound:
     print(f"\nError: Source table '{source_table_id_full}' or Model '{model_id_full}' not found.")
     print("Ensure the source table exists and the model was created successfully.")
except exceptions.PermissionDenied:
     print(f"\nError: Permission denied to create embeddings table or access data/model.")
     print("Ensure the service account has 'bigquery.tables.create', 'bigquery.tables.updateData', 'bigquery.tables.getData', and 'vertexai.endpoints.predict' permissions.")
except Exception as e:
    print(f"\nAn error occurred during embeddings table creation: {e}")
    print("Please check the source table, model, and permissions.")


print("\nBigQuery ML and Embeddings table creation script complete.")

BigQuery client initialized for project: qwiklabs-gcp-03-28c3125acb2b

Attempting to create/replace BigQuery ML model: qwiklabs-gcp-03-28c3125acb2b.genai_app.embeddings_model
Using connection: us.vertex-rag-connection
Pointing to Vertex AI endpoint: text-embedding-005
Model creation job 78dd3d8d-cb31-4458-806c-235e8686c86d initiated. Waiting for job to complete...
BigQuery ML model 'embeddings_model' created/replaced successfully in dataset 'genai_app'.

Attempting to create/replace embeddings table: alaska_dept_of_snow_faqs_with_embeddings
Using source data from: qwiklabs-gcp-03-28c3125acb2b.genai_app.alaska_dept_of_snow_faqs
Using model: qwiklabs-gcp-03-28c3125acb2b.genai_app.embeddings_model
Embeddings table creation job 2a5d704e-069e-4113-b81e-b6fd80df2551 initiated. Waiting for job to complete...
Embeddings table 'alaska_dept_of_snow_faqs_with_embeddings' created/replaced successfully in dataset 'genai_app'.
Successfully created embeddings table with 50 rows.

BigQuery ML and Embe

In [ ]:
# Documenting Example Questions and expected types in this section

validation_questions = {
    "q1": {"question": "What is the current snow forecast?", "type": "Current Weather"},
    "q2": {"question": "Are there any school closures?", "type": "Closures"},
    "q3": {"question": "What are the typical fees for snow removal? Our property value is $500,000", "type": "Fees"},
    "q4": {"question": "How does the Alaska snow department prioritize what to plow first?", "type": "General"},
    "q5": {"question": "Who makes the decisions on if a school will close?", "type": "Closures"},
}

#print(validation_questions["q1"]["question"])
# Gemini Tool - Annual Fee Calculation

What is the current snow forecast?


In [ ]:
# Main Runtime Script

from IPython import get_ipython
from IPython.display import display
import pandas as pd
from google.cloud import bigquery
from google import genai
from google.genai import types
import base64
import json
import requests
import vertexai
from vertexai.generative_models import GenerativeModel
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
    PairwiseMetric,
    PairwiseMetricPromptTemplate,
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
)
import datetime
import pytest

# Configuration
PROJECT_ID = "qwiklabs-gcp-03-28c3125acb2b"
LOCATION = "us-central1"

# For Weather API
headers = {
    'User-Agent': 'jason.lajeunesse@clearobject.com',
    'Accept': 'application/ld+json'
}

lat, lon = 63.5888, -154.4931  # Alaska Lat/Long
point_url = f'https://api.weather.gov/points/{lat},{lon}'

# BigQuery details for RAG
BQ_PROJECT_ID = PROJECT_ID # Usually the same as your main project ID
BQ_DATASET_ID = "genai_app"
BQ_SOURCE_TABLE_ID = "alaska_dept_of_snow_faqs"
BQ_EMBEDDINGS_TABLE_ID = f"{BQ_SOURCE_TABLE_ID}_with_embeddings" # alaska_dept_of_snow_faqs_with_embeddings
BQ_MODEL_ID = "embeddings_model" # The BQ ML model name

# Tooling: Fee Calculation
def calculate_snow_removal_fee(property_value: float) -> dict:
    # Snow removal calculation based upon property value

    # The fee is 0.1% of the property value per year
    annual_fee_rate = 0.001
    annual_fee = property_value * annual_fee_rate
    # Return as a dictionary so the model can easily use the structured output
    return {"annual_fee": round(annual_fee, 2)} # Round to 2 decimal places

# Fee calculation tool for Gemini, which utilizes property value
calculate_fee_tool = genai.types.Tool(
    function_declarations=[
        genai.types.FunctionDeclaration(
            name='calculate_snow_removal_fee',
            description='Calculates the annual snow removal fee based on the resident\'s property value.',
            parameters=genai.types.Schema(
                type="OBJECT",
                properties={
                    'property_value': genai.types.Schema(
                        type="NUMBER",
                        description='The resident\'s property value in US dollars.'
                    )
                },
                required=['property_value']
            )
        )
    ]
)

# Map function names to actual Python functions
available_functions = {
    'calculate_snow_removal_fee': calculate_snow_removal_fee,
}

# Map function names to actual Python functions
available_functions = {
    'calculate_snow_removal_fee': calculate_snow_removal_fee,
}

# Gemini Client & Base Generation Function
def get_gemini_client():
    """Initializes and returns the Gemini client."""
    return genai.Client(
        vertexai=True,
        project=PROJECT_ID,
        location=LOCATION,
    )

def generate_response_tools(user_input, system_instruction=None, tools=None, contents=None):

    # Updated generate response function to generate Gemini response, with streaming and tool outputs.
    # Can take existing conversation history (`contents`) and tools.

    client = get_gemini_client()
    model_name = "gemini-2.0-flash-001" # Using flash for speed

    # If contents is not provided, start a new conversation turn
    if contents is None:
         contents = [
            types.Content(
              role="user",
              parts=[types.Part.from_text(text=user_input)]
            )
         ]

    generate_content_config = types.GenerateContentConfig(
        temperature=0.3,
        top_p=0.95,
        max_output_tokens=8192,
        response_modalities=["TEXT"],
        system_instruction=[types.Part.from_text(text=system_instruction)] if system_instruction else None,
        tools=tools #now with tools
    )

    output_string = ""
    function_call_part = None # To store a potential function call from the model

    # The orchestration logic will handle the turns
    response = client.models.generate_content(
        model=model_name,
        contents=contents,
        config=generate_content_config,
    )

    # Process the response
    # Check if the model wants to call a function
    if response.candidates and response.candidates[0].content.parts:
        for part in response.candidates[0].content.parts:
            if part.function_call:
                function_call_part = part
                #print(f"Gemini requested function call: {function_call_part.function_call.name}")
                return {"function_call": function_call_part}
            if part.text:
                # Accumulate text for the final response (after tool use, or if no tool needed)
                output_string += part.text

    #print("Gemini returned text.")
    return {"text_response": output_string}

def generate_response(user_input, sys_instruction):

  # Original simple text generation function still used for classification/safety

  client = get_gemini_client()
  model = "gemini-2.0-flash-001"

  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text = user_input)
      ]
    )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 0.1, # Lower temp for classification
    top_p = 0.95,
    max_output_tokens = 128, # Keep output short for classification
    # response_modalities = ["TEXT"], # Text is default
    system_instruction=[types.Part.from_text(text=sys_instruction)],
  )

  output_string = ""

  # Using the original streaming approach for classification - it's fast and simple here
  # print("Calling Gemini for classification...")
  try:
      for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        output_string += chunk.text.replace('\n', ' ').replace('\r', '').strip() # Clean up output
      return output_string
  except Exception as e:
      print(f"Error during classification: {e}")
      return "ErrorClassifying" # Return a default error string

def classify_question(user_question):

  # Provides the classification of a question in one of the types of Employment, General Information, Emergency Services, or Tax Related

  sys_instruction = """
    You are a query classification agent that will provide an answer back on the type of user question provided to you.
    This is in support of another agent providing answers to residents of Alaska. But your focus is ONLY on classifying the type of query.

    These are the four possible results you should return (and please return a single word only for the best fitting category):
    Weather, Closures, Fees, or General (catch all bucket for Alaskan Snow Removal information that is not in the other 3 categories)
  """

  classification = generate_response(user_question, sys_instruction).replace('\n', ' ').replace('\r', '').rstrip()

  return classification

def is_valid(user_question):

  # Provides the classification of a question in one of the types of Employment, General Information, Emergency Services, or Tax Related

  sys_instruction = """
    You are chatbot designed to check for valid user input.

    User input is only valid if the question pertains to asking for information related to the Alaska Snow Department. Please note that
    a user may not explicitly ask about the Alaska snow department, but questions should relate to topics like general information about
    the Alaska Snow Department, current weather conditions, business or school closures, or fees related to the Alaskan Snow Department.
    All other topics are considered Invalid.

    Please respond with either Valid or Invalid
  """

  validity = generate_response(user_question, sys_instruction).replace('\n', ' ').replace('\r', '').rstrip()

  return validity

def is_safe(content):

  # Provides the classification of a question in one of the types of Employment, General Information, Emergency Services, or Tax Related

  sys_instruction = """
    You are chatbot designed to check for valid content, which could be either from a model result or an input prompt to a model.

    Any content provided that involves hate speach, physical or sexual harm, sexually explicit content, or dangerous material should be considered Unsafe.
    Content that does not contain these dangerous elements can be considered Safe.

    Please respond with whether the content is either Safe or Unsafe with a single word, Safe or Unsafe.
  """

  safety = generate_response(content, sys_instruction).replace('\n', ' ').replace('\r', '').rstrip()

  return safety

def retrieve_augmentations(user_input_query):

  # Retrieve the related Alaska Snow Removal FAQ infromation from BQ

   # GCP Details
  project_id = "qwiklabs-gcp-03-28c3125acb2b"
  client = bigquery.Client(project=project_id)

  # Initialize the BigQuery client
  client = bigquery.Client()

  # Step 1: Get the Top 5 Matching Results from BQ

  # Setup query to get top 5 results from BQ via similarity search, with the Vertex AI model connection generating the embedding of the user input question
  query = f"""
  SELECT query.query, base.question, base.answer
  FROM VECTOR_SEARCH(
    TABLE `qwiklabs-gcp-03-28c3125acb2b.genai_app.alaska_dept_of_snow_faqs_with_embeddings`, 'ml_generate_embedding_result',
    (
    SELECT text_embedding, content AS query
    FROM ML.GENERATE_TEXT_EMBEDDING(
    MODEL `qwiklabs-gcp-03-28c3125acb2b.genai_app.embeddings_model`,
    (SELECT @query_text AS content))
    ),
    top_k => 5, options => '{{"fraction_lists_to_search": 0.01}}')
  """

  # Configuration of the user input parameter
  job_config = bigquery.QueryJobConfig(
    query_parameters = [
        bigquery.ScalarQueryParameter("query_text", "STRING", user_input_query),
    ]
  )

  top5_results_df = client.query(query, job_config=job_config).to_dataframe()

  return top5_results_df

def get_weather_alaska_tonight():
  # Step 1: Get gridpoint info
  point_response = requests.get(point_url, headers=headers)
  forecast_url = point_response.json()['forecast']

  # Step 2: Get forecast from URL
  forecast_response = requests.get(forecast_url, headers=headers)
  forecast_data = forecast_response.json()

  # Step 3: Print each forecast period
  for period in forecast_data['periods']:
      if period['name'] =='Tonight':
        return period['detailedForecast']

def chat_with_tools_and_rag(user_input_query):

    # Handles the user query, classifying it and using either tools or RAG to generate a response.

    # Step 1: Initial Checks (Validity & Safety)
    if is_safe(user_input_query) == "Unsafe":
        return "Your query was flagged as potentially unsafe. Please rephrase your question."

    # Check input validity (if it's relevant to the Snow Dept)
    if is_valid(user_input_query) == "Invalid":
         # Check output safety of this canned response just in case
        if is_safe("Your query does not seem to be related to the Alaska Snow Department.") == "Unsafe":
             return "An unexpected issue occurred."
        return "Your query does not seem to be related to the Alaska Snow Department. Please ask about snow removal in Alaska."

    # Step 2: Classify the Query
    query_type = classify_question(user_input_query)

    # Step 3: Determine Response Strategy (Tools or RAG)
    final_response = ""

    if query_type == "Fees":
        #print("Query classified as 'Fees'. Attempting to use the fee calculation tool.")

        # System instruction for the tool-using model
        tool_system_instruction = """
        You are the helpful assistant for the Alaska Snow Department. The annual snow removal fee is calculated as 0.1% of the property value.
        Use the available tool 'calculate_snow_removal_fee' to get the fee based on the user's request.
        Once you have the result from the tool, state the annual fee clearly and politely to the user based on the property value they mentioned or that was used for calculation.
        If the user's prompt does not contain a property value, ask the user for their property value so you can calculate the fee.
        """

        # Conversation history starts with the user's turn
        conversation_history = [types.Content(role="user", parts=[types.Part.from_text(text=user_input_query)])]

        # First turn: Ask the model if it needs a tool
        response_turn1 = generate_response_tools(
            user_input=None, # Pass user_input as part of contents history
            system_instruction=tool_system_instruction,
            tools=[calculate_fee_tool], # Provide the tool definition
            contents=conversation_history # Pass the initial history
        )

        if "function_call" in response_turn1:
            # Model wants to call a function
            function_call_part = response_turn1["function_call"]
            function_name = function_call_part.function_call.name
            function_args = function_call_part.function_call.args # The arguments

            #print(f"Model requested function: {function_name} with args: {function_args}")

            if function_name in available_functions:
                # Call the actual Python function
                try:
                    # The args object is a protobuf message, convert to JSON then Python dict
                    args_dict = {key: value for key, value in function_args.items()}
                    tool_output_data = available_functions[function_name](**args_dict)

                    #print(f"Tool '{function_name}' executed. Output: {tool_output_data}")

                    # Prepare the function response for the model
                    function_response_part = types.Part.from_function_response(
                         name=function_name,
                         response=tool_output_data # Pass the dictionary output
                    )

                    # Add the model's function call and the function response to the history
                    conversation_history.append(function_call_part)
                    conversation_history.append(types.Content(role="function", parts=[function_response_part]))

                    # Second turn: Send the tool output back to the model to get the final response
                    response_turn2 = generate_response_tools(
                        user_input=None, # Already in history
                        system_instruction=tool_system_instruction, # Use the same instructions
                        tools=[calculate_fee_tool], # Still provide the tool definition
                        contents=conversation_history # Pass updated history
                    )

                    if "text_response" in response_turn2:
                        final_response = response_turn2["text_response"]
                    else:
                        final_response = "Could not get a final text response after tool execution."

                except Exception as e:
                    print(f"Error executing tool '{function_name}': {e}")
                    final_response = "An error occurred while calculating the fee."

            else:
                # Model requested a function that doesn't exist
                print(f"Model requested unknown function: {function_name}")
                final_response = f"The requested function '{function_name}' is not available."

        elif "text_response" in response_turn1:
            # Model didn't call a function, returned text directly (e.g., asking for property value)
            final_response = response_turn1["text_response"]
            #print("Model returned text instead of calling tool (likely asking for input).")

        else:
            final_response = "An unexpected response was received from the model during the fees process."

    elif query_type == "Weather":
        #print(f"Query classified as '{query_type}'. Performing weather lookup.")

        tonights_weather = get_weather_alaska_tonight()

        weather_system_instruction = f"""
          You are the helpful assistant for the Alaska Snow Department. Answer the user's question based on the weather information
          provided (for tonight's forecast in Alaska). Please note to the user that only tonight's forecast is available, if asking
          for a different timeframe.

          Toinight's weather forecast: {tonights_weather}
        """
        weather_response = generate_response_tools(
            user_input=user_input_query,
            system_instruction=weather_system_instruction,
            tools=None # No tools needed for RAG response generation
        )
        if "text_response" in weather_response:
             final_response = weather_response["text_response"]
        else:
             weather_response = "Could not generate a response based on the provided information."

    else:
        #print(f"Query classified as '{query_type}'. Performing RAG lookup.")

        # Perform RAG lookup for other categories
        augmentation_text = retrieve_augmentations(user_input_query)

        # Create reference prompt which will be appended to the system instructions
        prompt_qnas = "Please Use the Reference Question And Answers to answer the user's query: \n"

        for row in augmentation_text.itertuples():
            prompt_qnas = prompt_qnas + f"Question: {row.question}, Answer: {row.answer}\n"

        # System instruction for the RAG model
        rag_system_instruction = f"""
          You are the helpful assistant for the Alaska Snow Department. Answer the user's question based ONLY on the information provided.
          If the provided information does not contain the answer, state that you cannot find the answer in the provided information.
          Do not use outside knowledge.

          {prompt_qnas}
        """

        # Generate response using RAG and the retrieved context
        # Use generate_response_tools for more control, but pass user_input directly for non-tool cases
        rag_response = generate_response_tools(
            user_input=user_input_query,
            system_instruction=rag_system_instruction,
            tools=None # No tools needed for RAG response generation
        )
        if "text_response" in rag_response:
             final_response = rag_response["text_response"]
        else:
             final_response = "Could not generate a response based on the provided information."


    # Step 4: Final Safety Check of the generated response
    if is_safe(final_response) == "Unsafe":
        return "The generated response was flagged as potentially unsafe." # Return generic message

    return final_response

def test_calculate_snow_removal_fee():

    #Test for expected snow removal fee (0.1% of property value)

    try:
      property_value = 370000.0
      fee = calculate_snow_removal_fee(property_value)
      assert fee['annual_fee'] == 370.00

      print("PASSED: test_calculcated_snow_removal_fee() tests")

    except Exception as e:
      print(f"classify_question test FAILED: An error occurred - {e}")

def test_is_valid():

    #Test for expected safety results
    try:
      #Test 1
      user_input = "Tell me about puppies"
      result = is_valid(user_input)
      assert result == "Invalid"

      #Test 2
      user_input = "How does the Alaskan Snow Department impact school closures?"
      result = is_valid(user_input)
      assert result == "Valid"

      print("PASSED: is_valid() tests")

    except Exception as e:
      print(f"classify_question test FAILED: An error occurred - {e}")

def test_is_safe():

    #Test for expected safety results
    try:
      #Test 1
      user_input = "Do something violent!"
      result = is_safe(user_input)
      assert result == "Unsafe"

      #Test 2
      user_input = "How does the Alaskan Snow Department impact school closures?"
      result = is_safe(user_input)
      assert result == "Safe"

      print("PASSED: is_safe() tests")

    except Exception as e:
      print(f"classify_question test FAILED: An error occurred - {e}")

def test_classify_questions():

    #Test for classifying - Weather, Closures, Fees, or General
    try:
      #Test 1
      user_input = "How is the weather looking?"
      result = classify_question(user_input)
      assert result == "Weather"

      #Test 2
      user_input = "How does the Alaskan Snow Department impact school closures?"
      result = classify_question(user_input)
      assert result == "Closures"

      #Test 3
      user_input = "What are the typical fees for snow removal in Alaska? Our property value is $370,000?"
      result = classify_question(user_input)
      assert result == "Fees"

      #Test 4
      user_input = "How long has the Alaskan Department of Snow been around?"
      result = classify_question(user_input)
      assert result == "General"

      print("PASSED: classify_question() tests")

    except Exception as e:
      print(f"classify_question test FAILED: An error occurred - {e}")

def run_tests():

  # Run tests for the snow removal tool & key agent functions (safe, valid, classification)
  test_classify_questions()
  test_is_safe()
  test_is_valid()
  test_calculate_snow_removal_fee()

def run_eval_chat_with_tools_and_rag():

  # Classification Questions Evaluation for groundedness and coherence
  user_questions = [
      "What are the typical fees for snow removal in Alaska? Our property value is $370,000",
      "How long has the Alaskan Department of Snow been around?",
      "What is the fee?",
      "Tell me about cats",
      "How is the weather looking tomorrow?",
      "How is the weather looking?"
  ]

  responses =[]

  for prompt in user_questions:
    response = chat_with_tools_and_rag(prompt)
    responses.append(response)
    print("\nUser Query: " + prompt + "\nResponse: " + response)

  # Eval Dataset Setup
  eval_dataset = pd.DataFrame({
    "prompt": responses
  })

  eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[
      MetricPromptTemplateExamples.Pointwise.FLUENCY,
      MetricPromptTemplateExamples.Pointwise.COHERENCE
    ],
    experiment="main-genai-app-runtime",
  )

  model = GenerativeModel(
    "gemini-2.0-flash-001",
    generation_config={
        "temperature": 0,
        "top_p": 0.4,
    },
  )

  run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  eval_result = eval_task.evaluate(
    model=model,
    experiment_run_name=f"apt-gen-{run_ts}"
  )

  eval_result.metrics_table
  print(eval_result.metrics_table)

# Main execution
def main():

  # Run Tests for key functions
  run_tests()

  # Run evaluation of the main GenAI runtime
  run_eval_chat_with_tools_and_rag()

if __name__ == "__main__":
   main()

PASSED: classify_question() tests
PASSED: is_safe() tests
PASSED: is_valid() tests
PASSED: test_calculcated_snow_removal_fee() tests

User Query: What are the typical fees for snow removal in Alaska? Our property value is $370,000
Response: The annual snow removal fee for your property, valued at $370,000, is $370.


User Query: How long has the Alaskan Department of Snow been around?
Response: The Alaska Department of Snow (ADS) was established in 1959, coinciding with Alaska’s admission as a U.S. state.


User Query: What is the fee?
Response: I can calculate the annual snow removal fee for you. What is your property value?


User Query: Tell me about cats
Response: Your query does not seem to be related to the Alaska Snow Department. Please ask about snow removal in Alaska.

User Query: How is the weather looking tomorrow?
Response: I am sorry, I only have access to tonight's weather forecast. Tonight's forecast is: Isolated rain showers before 7pm. Mostly cloudy, with a low around 

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/416100490526/locations/us-central1/metadataStores/default/contexts/main-genai-app-runtime-apt-gen-20250429-205502 to Experiment: main-genai-app-runtime


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'model_name': 'publishers/google/models/gemini-2.0-flash-001', 'temperature': 0, 'top_p': 0.4}
INFO:vertexai.evaluation._evaluation:Generating a total of 6 responses from Gemini model gemini-2.0-flash-001.
100%|██████████| 6/6 [00:01<00:00,  4.07it/s]
INFO:vertexai.evaluation._evaluation:All 6 responses are successfully generated from Gemini model gemini-2.0-flash-001.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 1.489505163001013 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 12 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 12/12 [00:12<00:00,  1.01s/it]
INFO:vertexai.evaluation._evaluation:All 12 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:12.113433942999109 seconds


                                              prompt  \
0  The annual snow removal fee for your property,...   
1  The Alaska Department of Snow (ADS) was establ...   
2  I can calculate the annual snow removal fee fo...   
3  Your query does not seem to be related to the ...   
4  I am sorry, I only have access to tonight's we...   
5  Tonight's forecast is mostly cloudy with a low...   

                                            response  \
0  Okay, that means the snow removal fee is 0.1% ...   
1  This is a clear and concise statement of fact....   
2  I do not have a property value as I am an AI. ...   
3  Okay, I understand.\n\n**What are the common m...   
4  Okay, thanks for the information. So, to summa...   
5  Okay, here's a summary of tonight's weather fo...   

                                 fluency/explanation  fluency/score  \
0  The response is completely fluent because it d...            5.0   
1  The response is fluent, employing sophisticate...            5.0   
2